In [1]:
#inititalize the notebook
import os
import textwrap
import pymongo
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()

openai_client = AzureOpenAI(
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = os.getenv("OPENAI_API_VERSION"),
    azure_endpoint = os.getenv("AZURE_OPENAI_API_BASE")
)

mongo_user = os.getenv("COSMOS_MONGO_USER")
mongo_pwd = os.getenv("COSMOS_MONGO_PWD")
mongo_server = os.getenv("COSMOS_MONGO_SERVER")
mongo_conn = f"mongodb+srv://{mongo_user}:{mongo_pwd}@{mongo_server}?tls=true&authMechanism=SCRAM-SHA-256&retrywrites=false&maxIdleTimeMS=120000"


embeddings_deployment_name = "text-embedding-ada-002"
chat_deployment_name = "gpt-4"

print("*** init done! ***")


*** init done! ***


Hoe ziet een vector er uit?

In [2]:
word = "cat"

embeddings = openai_client.embeddings.create(input = word, model=embeddings_deployment_name)

embeddingData = embeddings.data[0].embedding

print("De vector voor de het woord: " + word)
print("Het aantal dimensies in de vector is: " + str(len(embeddingData)))
print("Dit zijn de waardes van de vector: ")
print(embeddingData)

De vector voor de het woord: cat
Het aantal dimensies in de vector is: 1536
Dit zijn de waardes van de vector: 
[-0.0070945825427770615, -0.017328109592199326, -0.009644086472690105, -0.03070768155157566, -0.012548675760626793, 0.003105211304500699, -0.005113212391734123, -0.04121817275881767, -0.014629469253122807, -0.021376069635152817, 0.019231360405683517, 0.05087646469473839, -0.0012907310156151652, 0.0024855893570929766, -0.03840590640902519, -0.006089693866670132, 0.0355084203183651, -0.004697763826698065, 0.0023630852811038494, -0.01342928409576416, -0.01891888678073883, 0.009019138291478157, 0.015893569216132164, -0.008713766001164913, -0.014672079123556614, 0.007233065087348223, 0.013031589798629284, -0.013365369290113449, 0.002858427818864584, 0.004861102905124426, 0.0040266546420753, -0.01677417755126953, -0.015850959345698357, -0.04306461289525032, -0.027242060750722885, -0.004278764594346285, 0.0080533092841506, -0.009984967298805714, 0.022015219554305077, -0.009040444158

Vergelijk de semantische gelijkheid van woorden

We hebben een lijst van woorden en een woord dat we willen vergelijken.

* Bereken een vector van ieder woord in de lijst
* Bereken een vector van het woord dat we willen vergelijken
* Bepaal de gelijkheid (score) met een dotproduct algoritme (er zijn meerdere algoritmes om dit te doen)

In [6]:
### DotProduct is a simple algorithm to calculate simularity between two vectors
### Multiply each element in vector1 with the corresponding element in vector2
### Add the results together for each pair of elements
### The result is the dot product of the two vectors
### https://www.mathsisfun.com/algebra/vectors-dot-product.html

def dotproduct(vector1, vector2):
    #return sum((a*b) for a, b in zip(vector1, vector2))
    #get the length of the vector1 
    length = len(vector1)

    #check if the length of the vector2 is the same as vector1
    if length != len(vector2):
        print("The vectors are not the same length")
        return

    #iterate over the vector based on the length of the vector
    sum = 0

    for i in range(length):
        #multiply the vectors and add them to the sum
        sum += vector1[i] * vector2[i]

    return sum

##### end of the dotproduct function #####

words = ['cat', 
         'hamburger', 
         'computer', 
         'server',
         'dog', 
         'pizza', 
         'laptop', 
         'horse', 
         'car',
         'steak', 
         'truck',
         'pickup']

word = 'cat'

#create a list of embeddings for each word
wordListEmbedding = []
for w in words:
    response = openai_client.embeddings.create(input = w, model=embeddings_deployment_name).data[0].embedding
    wordListEmbedding.append(response)

#create an embedding for the word we want to compare
wordEmbedding = openai_client.embeddings.create(input = word, model=embeddings_deployment_name).data[0].embedding

#calculate the dot product between the word we want to compare and each word in the list
dotProductList = []
for w in wordListEmbedding:
    dotProductList.append(dotproduct(wordEmbedding, w))

#sort the dotProductList
sortedDotProductList = sorted(dotProductList, reverse=True)

#find the index of the words with the highest dot product
indexList = []
for i in sortedDotProductList:
    indexList.append(dotProductList.index(i))

#display the top 3 words based on indexlist
print("De top 3 woorden vergelijkbaar met '" + word + "' zijn: ")
print(f"'{words[indexList[0]]}' dotproduct score: + {str(sortedDotProductList[0])}")
print(f"'{words[indexList[1]]}' dotproduct score: + {str(sortedDotProductList[1])}")
print(f"'{words[indexList[2]]}' dotproduct score: + {str(sortedDotProductList[2])}")






De top 3 woorden vergelijkbaar met 'cat' zijn: 
'cat' dotproduct score: + 1.000000016120353
'dog' dotproduct score: + 0.8635463511813951
'car' dotproduct score: + 0.845611581735055


De vector kun je ook berekenen over een zin.

In dit geval nemen we user prompt uit het laatste voorbeeld dat we hebben gezien bij "prompts"

In [7]:
sentence = "Wanneer is de open dag?"

embeddings = openai_client.embeddings.create(input = sentence, model=embeddings_deployment_name)

embeddingData = embeddings.data[0].embedding

print("De vector voor de de zin: " + sentence)
print("Het aantal dimensies in de vector is: " + str(len(embeddingData)))
print("Dit zijn de waardes van de vector: ")
print(embeddingData)

De vector voor de de zin: Wanneer is de open dag?
Het aantal dimensies in de vector is: 1536
Dit zijn de waardes van de vector: 
[0.009035943076014519, -0.03067838028073311, 0.004121601581573486, -0.019322223961353302, -0.007837166078388691, 0.00893282238394022, 0.004411628469824791, 0.004801553208380938, -0.007257112767547369, 0.004179607145488262, 0.007850056514143944, 0.01465601660311222, -0.004872448742389679, -0.017053570598363876, -0.025496570393443108, 0.003744567045941949, 0.03897958993911743, -0.024671604856848717, 0.006812405306845903, -0.01209733635187149, -0.005388051737099886, 0.004227944649755955, -0.0027214172296226025, 0.0030968408100306988, -0.014114633202552795, -0.002281543333083391, 0.009042387828230858, -0.01573878340423107, 0.004282727837562561, -0.014166193082928658, 0.015777451917529106, -0.011697744019329548, -0.014501335099339485, 0.0025393448304384947, 0.019051531329751015, -0.0067995148710906506, 0.006909080781042576, 0.022042030468583107, 0.0198893863707780

laten we eens kijken of we de meest relevante pagina kunnen vinden om de vraag te beantwoorden.

* We gaan een vector maken van de vraag
* we gaan voor iedere pagina een vector maken
* We gaan de meest relevante pagina selecteren en in de prompt meegeven

In [8]:
#lees de pagina's van de website
page1 = open("../docs/page-0.md", "r").read()
page2 = open("../docs/page-1.md", "r").read()
page3 = open("../docs/page-2.md", "r").read()

pages = [page1, page2, page3]

myDataQuestion = "Wanneer is de open dag?"

pagesListEmbedding = []
for p in pages:
    response = openai_client.embeddings.create(input = p, model=embeddings_deployment_name).data[0].embedding
    pagesListEmbedding.append(response)

#create an embedding for the word we want to compare
questionEmbedding = openai_client.embeddings.create(input = myDataQuestion, model=embeddings_deployment_name).data[0].embedding

#calculate the dot product between the word we want to compare and each word in the list
dotProductList = []
for p in pagesListEmbedding:
    dotProductList.append(dotproduct(questionEmbedding, p))

#sort the dotProductList
sortedDotProductList = sorted(dotProductList, reverse=True)

#find the index of the words with the highest dot product
indexList = []
for i in sortedDotProductList:
    indexList.append(dotProductList.index(i))

#augment the prompt with the most relevant page
myDataResponse = openai_client.chat.completions.create(
    model = chat_deployment_name,
    temperature=0.0,
    messages=[
        {
            "role": "user",
            "content": myDataQuestion
        },
        {
            "role": "system",
            "content" : f"""
                Je bent een behulpzame assistent
                Je geeft alleen antwoord op basis van de data die je hebt gekregen
                Indien je het antwoord niet kunt vinden, 
                zeg je dat je het antwoord niet weet
                
                ## data
                {pages[indexList[0]]}

                """
        }
    ]
)


lines = textwrap.wrap(myDataResponse.choices[0].message.content, width=80)
for line in lines:
    print(line)

print("----")
print("Hier wat info over de tokens die je hebt gebruikt:")
print(myDataResponse.usage)



De open dag is op zaterdag 27 januari, van 10.00 tot 15.00 uur in Heerlen,
Maastricht en Sittard.
----
Hier wat info over de tokens die je hebt gebruikt:
CompletionUsage(completion_tokens=39, prompt_tokens=1911, total_tokens=1950)


Dit is al een stuk beter!

Maar hebben we het hele document wel nodig?

In [9]:
# create an array of files in the chunks directory
files = os.listdir("../chunks")
pages = []

for file in files:
    page = open(f"../chunks/{file}", "r").read()
    pages.append(page)

pagesListEmbedding = []
for p in pages:
    response = openai_client.embeddings.create(input = p, model=embeddings_deployment_name).data[0].embedding
    pagesListEmbedding.append(response)

#create an embedding for the word we want to compare
questionEmbedding = openai_client.embeddings.create(input = myDataQuestion, model=embeddings_deployment_name).data[0].embedding

#calculate the dot product between the word we want to compare and each word in the list
dotProductList = []
for p in pagesListEmbedding:
    dotProductList.append(dotproduct(questionEmbedding, p))

#sort the dotProductList
sortedDotProductList = sorted(dotProductList, reverse=True)

#find the index of the words with the highest dot product
indexList = []
for i in sortedDotProductList:
    indexList.append(dotProductList.index(i))

#augment the prompt with the most relevant page
myDataResponse = openai_client.chat.completions.create(
    model = chat_deployment_name,
    temperature=0.0,
    messages=[
        {
            "role": "user",
            "content": myDataQuestion
        },
        {
            "role": "system",
            "content" : f"""
                Je bent een behulpzame assistent
                Je geeft alleen antwoord op basis van de data die je hebt gekregen
                Indien je het antwoord niet kunt vinden, 
                zeg je dat je het antwoord niet weet
                
                ## data
                {pages[indexList[0]]}

                """
        }
    ]
)


lines = textwrap.wrap(myDataResponse.choices[0].message.content, width=80)
for line in lines:
    print(line)

print("----")
print("Hier wat info over de tokens die je hebt gebruikt:")
print(myDataResponse.usage)



De open dag is op zaterdag 27 januari tussen 10.00 en 15.00 uur.
----
Hier wat info over de tokens die je hebt gebruikt:
CompletionUsage(completion_tokens=25, prompt_tokens=167, total_tokens=192)


Weer een verbetering! Minder tokens en toch een antwoord!

Embeddings berekenen kost echter ook tokens

In [10]:
response = openai_client.embeddings.create(input = pages[indexList[0]], model=embeddings_deployment_name)
response.usage

Usage(prompt_tokens=86, total_tokens=86)

Vectoren berekenen hoeven we niet iedere keer te doen.

We kunnen de vectoren ook opslaan en hergebruiken.

Hier hebben we een vector database voor nodig. 

Er zijn verschillende vector databases. 

We kiezen hier voor MongoDb. In dit geval gebruiken Cosine Simularity als algoritme. Dit algoritme is ingebouwd in de database om gelijkheid te berekenen.

In [11]:
databaseName = "hszuyd"
collectionName = "webpagecollection"

mongo_client = pymongo.MongoClient(mongo_conn)
db = mongo_client[databaseName]
collection = db[collectionName]

if collectionName not in db.list_collection_names():
    db.create_collection(collectionName)
else:
    #delete all documents in the collection so we can rerun this script
    collection.delete_many({})


db.command({
  'createIndexes': f'{collectionName}',
  'indexes': [
    {
      'name': 'VectorSearchIndex',
      'key': {
        "contentVector": "cosmosSearch"
      },
      'cosmosSearchOptions': {
        'kind': 'vector-ivf',
        'numLists': 1,
        'similarity': 'COS',
        'dimensions': 1536
      }
    }
  ]
})

files = os.listdir("../chunks")
pages = []
filesUploaded = 0
for file in files:
    page = open(f"../chunks/{file}", "r").read()
    response = openai_client.embeddings.create(input = page, model=embeddings_deployment_name).data[0].embedding
    
    filetoStore = {
        "filename": file,
        "content": page,
        "contentVector": response
    }

    collection.insert_one(filetoStore)
    filesUploaded += 1

print(f"{filesUploaded} bestanden en vectoren opgeslagen in de database")


24 bestanden en vectoren opgeslagen in de database


We kunnen nu een database query uitvoeren met de vector van de vraag als input (dit is geen SQL query, maar een vector search opdracht).

De output is een lijst met relevante documenten die in de database zijn opgeslagen.

In [13]:
def vector_search(vector, num_results):
    pipeline = [
        {
            '$search': {
                "cosmosSearch": {
                    "vector": vector,
                    "path": "contentVector",
                    "k": num_results 
                },
                "returnStoredSource": True }},
        {'$project': { 'similarityScore': { '$meta': 'searchScore' }, 'document' : '$$ROOT' } }
    ]
    results = collection.aggregate(pipeline)
    return results

myDataQuestion = "Wanneer is de open dag?"
#create an embedding for the question we want to compare
questionEmbedding = openai_client.embeddings.create(input = myDataQuestion, model=embeddings_deployment_name).data[0].embedding

#zoek 3 relevante pagina's op basis van de vraag
results = vector_search(questionEmbedding, 23)

print("De top 3 resultaten zijn: ")
for result in results:
    print(result["document"]["filename"] + " - " + str(result["similarityScore"]))


De top 3 resultaten zijn: 
hbo-ict-2.md - 0.8554227097206687
hbo-ict-19.md - 0.8442692531897418
hbo-ict-16.md - 0.8275229725269576
hbo-ict-6.md - 0.8103652822535952
hbo-ict-21.md - 0.7955045462994585
hbo-ict-15.md - 0.7932169211118972
hbo-ict-0.md - 0.7869234554126288
hbo-ict-7.md - 0.7846934199333414
hbo-ict-1.md - 0.7834346343771726
hbo-ict-3.md - 0.7828755429758018
hbo-ict-8.md - 0.7828755429758018
hbo-ict-17.md - 0.7828098327857569
hbo-ict-9.md - 0.7800605195706705
hbo-ict-13.md - 0.7761628956989421
hbo-ict-20.md - 0.7754770188955101
hbo-ict-11.md - 0.7724404010247761
hbo-ict-18.md - 0.771296317268296
hbo-ict-999.md - 0.7700350969925247
hbo-ict-12.md - 0.769121463508257
hbo-ict-5.md - 0.7686168918425939
hbo-ict-4.md - 0.7582087228547489
hbo-ict-10.md - 0.7541152844704733
hbo-ict-14.md - 0.7528765496162767
